# Load a [TopCat](https://www.star.bris.ac.uk/~mbt/topcat/)-like catalogue

A format quite used in the field are catalogues of objects for which an header lists the entries present in the following rows, typically listing a series of sources and relative properties.

The first two lines of a catalogue formatted in such a way are shown below:

In [ ]:
input_file = 'data/dustpedia_z0.dat'
!head -2 $input_file

> Credits for the data in the above catalogue goes to the [DustPedia Collaboration](http://dustpedia.astro.noa.gr/) and to [Casasola et al. (2020, A&A, 633, 100)](https://www.aanda.org/articles/aa/pdf/2020/01/aa36665-19.pdf) in particular.

Note that, as already mentioned, the first line lists all the fields of the following lines, starting from the second line, each hosts data from a different source.

In order to automatise and ease loading such catalogues in a format that is compliant with the requirements of the library, a specific function for the conversion of catalogues into dictionary is made available and can be imported by calling

In [ ]:
from galapy.internal.utils import cat_to_dict

The documentation of the function can be retrieved with:

In [ ]:
help(cat_to_dict)

The purpose is to read from catalogue files a list of bands, fluxes and associated errors into a nested (2-levels) dictionary.

Some features:

- the comment character ``'#'`` at the beginning of the header line is optional. If it is present though, a space at the beginning of each following line is necessary.
- the entries are automatically converted into floats (except for the column passed as ``id_field``). All the entries that cannot be converted into floats (except for the column passed as ``id_field``) will be skipped and a warning message will be print on screen.
- the system automatically interprets as separators tabs, spaces and commas (i.e. ``'1 2,3\t4'`` is automatically interpreted as 4 different entries: ``1``, ``2``, ``3`` and ``4``). Note, though, this also means not to insert spaces into the ``id_field`` entry.

Let's try it out:

In [ ]:
catalogue = cat_to_dict( 
    input_file, id_field='name', err_field='_err', 
    meta_fields = ['redshift', 'redshift_err'], 
    skip_fields = ['ra', 'dec', 'semimaj_arcsec', 'axial_ratio', 'pos_angle', '*_flag']
)

We have made the following choices, based on the format of the file:

- ``id_field = 'name'`` the column named ``'name'`` is the identification name of the source.
- ``err_field = '_err'`` all fields containing the ``'_err'`` string are interpreted as errors on fluxes.
- the ``meta_fields`` argument lists all the entries that have to be read but not interpreted as a flux nor an error on a flux.
- the ``skip_fields`` argument lists all the entries that have to be ignored when reading the file. 

Note also that we have used a magic character in ``'*_flag'``, this tells the system to apply the specified behaviour to all fields that end with the string ``'_flag'``.

The object we named ``catalogue`` is a standard python dictionary, we can list the sources stored with:

In [ ]:
list(catalogue)

Each source is a subdictionary with the following entries:

In [ ]:
list(catalogue['NGC3898'])

> (we choose to list the entries of the NGC3898 but the same would have been shown for any other source in the catalogue)

As you can see, we have 3 default entries:

- ``bands``: a list of band (filter) names 
- ``fluxes``: a list of fluxes associated to the bands listed in ``bands``
- ``errors``: a list of errors associated to the flux measurement listed in ``fluxes``

**NOTE THAT GalaPy USES MILLIJANSKY AS FLUX UNIT, THEREFORE, IF THE FLUXES IN THE CATALOGUE ARE NOT IN MILLIJANSKY, CONVERT THEM AS SOON AS YOU LOAD THEM!**

Additionally, the system also loaded the ``redshift`` and associated error as meta-fields, as requested when calling the function.

## Let's see what we have loaded

First of all, we can check what bands we have measured fluxes, this can be by calling

In [ ]:
catalogue['NGC3364']['bands']

Conveniently, we have stored the band names with the same format accepted by GalaPy, i.e.:

> **Experiment[.Instrument].BandName**

and **all the filters** are present in the data-base.

> Note that you can check what filters are present in the database by calling 
>```python
>from galapy.PhotometricSystem import print_filters
>print_filters()
>```

Therefore, we can build a photometric-system for each of object in the catalogue:

In [ ]:
from galapy.PhotometricSystem import PMS

for obj in catalogue.values() :
    obj['pms'] = PMS(*obj['bands'])

In the catalogue above, upper limits have been marked as fluxes with negative errors. Having a negative value for the error is not liked by the system, we want to change that and keep track of the *"is an upper-limit"* flag.

We are therefore saving a boolean array for each object pointing out where the errors were negative in the original catalogue **and** we are replacing the negative errors with the corresponding value of flux (i.e. we are assuming the flux is 1 times the error):  

In [ ]:
for key in catalogue :
    obj = catalogue[key]
    obj['uplims'] = obj['errors']<0.0
    obj['errors'][obj['uplims']] = obj['fluxes'][obj['uplims']]
    print(f"object {key} has {obj['uplims'].sum()} non-detections")

Functions of the ``galapy.analysis.plot`` allow to generate formatted plots to inspect these datasets:

In [ ]:
from galapy.analysis import plot as gplot

# build a matplotlib figure and axes array with the internal pyplot format 
fig, axes = gplot.plt.subplots(2,2,figsize=(10,6), constrained_layout=True)

# Loop on the 4 objects and axes
for ax, key in zip(axes.flatten(), catalogue) :
    
    # set a title for the axes
    ax.set_title(key)
    
    # extract object from catalogue
    obj = catalogue[key]
    
    # set the image layout (with axis limits)
    ax = gplot.sed_layout(
        redshift=obj['redshift'], frame='rest', ax = ax, 
        xlim=(1.e+3, 1.e+7), ylim=(1.e-4, 1.e+3)
    )
    
    # plot the fluxes
    _ = gplot.sed_obs(
        obj['pms'].lpiv, obj['fluxes'], obj['errors'], 
        lo = obj['uplims'],
        redshift = obj['redshift'], frame = 'rest', ax = ax
    )

Note that the ``galapy.analysis.plot`` module also contains a function for showing the photometric system used:

In [ ]:
fig, ax = gplot.plt.subplots(1,1,figsize=(12,3), constrained_layout=True)
_ = gplot.photometric_system(catalogue['NGC3364']['pms'], ax=ax)